# **Project Movie Recommendation System**
**Nama:** Rahmah Fauziah

**ID:** MC299D5X1751

**Dataset yang digunakan:** https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system

## **Data Understanding**
Tahap ini dilakukan untuk memahami informasi dari data yang dimiliki.

Pada tahap ini melakukan import library yang akan digunakan meliputi pandas, TfidfVectorizer, dan cosine_similarity.

In [41]:
# Import library
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Import module yang disediakan google colab untuk kebutuhan upload file `kaggle.json`.

In [ ]:
# Import module yang disediakan google colab untuk kebutuhan upload file
from google.colab import files
files.upload()

Tahapan di bawah untuk mengelola kredensial API Kaggle supaya bisa melakukan download dataset dari Kaggle.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Melakukan download dataset yang berasal dari Kaggle dan melakukan unzip file.

In [ ]:
!kaggle datasets download -d parasharmanas/movie-recommendation-system
!unzip movie-recommendation-system.zip

**Output:** diketahui bahwa terdapat 2 file csv yaitu movies.csv dan ratings.csv.

Melakukan pembacaan data-data dengan menggunakan fungsi `pandas.read_csv`.

In [ ]:
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')

print('Jumlah data informasi film unik: ', len(movies['movieId'].unique()))
print('Jumlah data informasi pengguna unik: ', len(ratings['userId'].unique()))

**Output:** menunjukkan jumlah data pada masing-masing file csv dengan mengambil sampel berdasarkan kolom unik yaitu `movieId` dan `userId`. Untuk file `movies.csv` jumlah data terkait informasi film unik berdasarkan `movieId` sebanyak 62423. Untuk `ratings.csv` menurut `userId` jumlahnya sebesar 162541.

## **Exploratory Data Analysis**
Tahap ini dilakukan untuk mengetahui investigasi awal untuk melakukan analisis karakteristik dari dataset.

### **Univariate Analysis**
Tahapan ini untuk melakukan analisis dan eksplorasi setiap variabel pada data. Adapun variabel nya meliputi 2 macam yaitu movies dan ratings.

movies: merupakan data yang berisikan informasi terkait film yang meliputi kolom movieId, title, dan genres.

ratings: merupakan data rating dari film berisikan kolom userId, movieId, rating, dan timestamp.

#### **Movies Variabel**
Melakukan eksplorasi pada variabel movies yaitu informasi terkait film.

Pada tahap ini untuk mengetahui tipe data setiap kolom serta jumlah entri non-null pada variabel movies.

In [ ]:
movies.info()

**Output:** dapat diketahui bahwa file movies.csv memiliki 62423 entri dan 3 kolom. Terdapat tiga kolom, yaitu movieId, title, dan genres.
- movieId merupakan Id film yang bertipe data `int64`.
- title merupakan judul film yang memiliki tipe data `object`.
- genres merupakan genre pada film yang memiliki tipe data `object`.

Pada tahap di bawah ini untuk melakukan eksplorasi isi kolom pada data movies melalui fungsi `head()` dengan menampilkan lima baris pertama.

In [ ]:
movies.head()

**Output:** menampilkan tiga kolom yaitu movieId, title, dan genres yang berisikan data setiap kolom. Untuk movieId berisikan data nilai unik setiap film, title berisikan judul film dilengkapi dengan tahun perilisannya, dan genres berisikan macam-macam genre dari film dengan dipisahkan oleh tanda `(|)`.

#### **Ratings Variabel**
Melakukan eksplorasi pada variabel ratings yaitu informasi terkait rating pada film.

Pada tahap ini untuk mengetahui tipe data setiap kolom serta jumlah entri non-null pada variabel ratings.

In [ ]:
ratings.info()

Output: diketahui bahwa file ratings.csv memiliki 25000095 baris dan 4 kolom. Terdapat empat kolom, yaitu userId, movieId, rating, dan timestamp.
- userId merupakan Id pengguna yang bertipe data `int64`.
- movieId merupakan Id film bertipe data `int64`.
- rating merupakan nilai rating film yang bertipe data `float64`.
- timestamp merupakan waktu pemberian rating yang bertipe data `int64`.

Pada tahap di bawah ini untuk melakukan eksplorasi isi kolom pada data ratings melalui fungsi `head()` dengan menampilkan lima baris pertama.

In [ ]:
ratings.head()

Output: menampilkan empat kolom yaitu userId, movieId, rating, dan timestamp yang berisikan data setiap kolom. Untuk userId berisikan data nilai unik pengguna, movieId berisikan data nilai unik setiap film, rating berisikan nilai rating yang diberikan pengguna, dan timestamp berisikan waktu pemberian rating.

Pada tahap ini untuk mengecek deskripsi statistik data ratings dengan fungsi `describe()`. Fungsi describe() memberikan informasi statistik pada masing-masing kolom, antara lain:
- Count  adalah jumlah sampel pada data.
- Mean adalah nilai rata-rata.
- Std adalah standar deviasi.
- Min yaitu nilai minimum setiap kolom.
- 25% adalah kuartil pertama. Kuartil adalah nilai yang menandai batas interval dalam empat bagian sebaran yang sama.
- 50% adalah kuartil kedua, atau biasa juga disebut median (nilai tengah).
- 75% adalah kuartil ketiga.
- Max adalah nilai maksimum.

In [ ]:
ratings.describe()

**Output:** diketahui bahwa nilai maksimum rating adalah 5 dan nilai minimum adalah 0,5. Untuk rata-rata pemberian rating oleh pengguna berada di kisaran 3,5. Hal ini menunjukkan bahwa pengguna memberikan rating yang cukup tinggi untuk film nya. Jumlah pengguna dan film memiliki nilai maksimum 1.625410e+05 dan  dan 2.091710e+05 dan nilai minimum adalah 1.000000e+00.

Tahapan ini untuk meninjau banyaknya pengguna yang memberikan rating dan banyaknya jumlah rating.

In [ ]:
print('Jumlah userId: ', len(ratings['userId'].unique()))
print('Jumlah data rating: ', len(ratings))

**Output:** diketahui bahwa jumlah pengguna yang memberikan rating berdasarkan kolom unik yaitu `userId` sebanyak 162541 sedangkan untuk jumlah rating sebanyak 25000095.

### **Pengecekan nilai missing value**
Tahap ini untuk melakukan pengecekan terhadap kolom data movies dan ratings apakah memiliki nilai missing values.

In [ ]:
# Cek missing value dengan fungsi isnull()
movies.isnull().sum()

In [ ]:
# Cek missing value dengan fungsi isnull()
ratings.isnull().sum()

**Output:** diketahui bahwa pada data movies dan ratings setiap kolomnya tidak memiliki nilai missing value sehingga menghasilkan keluaran 0.

### **Pengecekan duplikasi data**
Melakukan identifikasi pada data movies dan ratings apakah terdapat baris yang memiliki duplikasi.

In [ ]:
# Mengidentifikasi baris duplikat
duplicates = movies.duplicated()

print("Baris duplikat:")
print(movies[duplicates])

In [ ]:
# Mengidentifikasi baris duplikat
duplicates = ratings.duplicated()

print("Baris duplikat:")
print(ratings[duplicates])

**Output:** diketahui bahwa tidak ada baris yang memiliki duplikasi pada data movies dan ratings ditunjukkan dengan keluaran Baris duplikat: Empty DataFrame.

## **Data Preparation**
Tahapan ini adalah tahapan untuk mempersiapkan data sebelum tahap modelling.

Pada tahap ini melakukan perubahan tipe data timestamp menjadi datetime melalui fungsi `pd.to_datetime`.

In [ ]:
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.head()

**Output:** menampilkan hasil perubahan data pada kolom timestamp menjadi bentuk datetime.

Tahapan ini untuk melakukan penggabungan data ratings dengan movies berdasarkan nilai movieId.

In [ ]:
# Menggabungkan dataframe ratings dengan movies berdasarkan nilai movieId
movies_info = pd.merge(ratings, movies , on='movieId', how='left')
movies_info

**Output:** menghasilkan penggabungan data ratings dengan movies yang di mana kolom title dan genres pada data movies ditambahkan ke sebelah kiri pada kolom data ratings, sehingga menghasilkan 6 kolom dan 25000095 baris.

Pada tahap ini melakukan proses pengurutan movies_info berdasarkan movieId kemudian memasukkannya ke dalam variabel fix_movies.

In [ ]:
# Mengurutkan movies_info berdasarkan movieId kemudian memasukkannya ke dalam variabel fix_movies
fix_movies = movies_info.sort_values('movieId', ascending=True)
fix_movies

**Output:** menghasilkan data yang berurutan berdasarkan kolom movieId melalui teknik **Ascending**atau mengurutkan nilai dari yang terkecil ke terbesar, di mana memiliki jumlah baris 25000095 dan 6 kolom.

Tahapan untuk melakukan pengecekan jumlah dari fix_movies berdasarkan movieId.

In [ ]:
# Mengecek berapa jumlah fix_movies
len(fix_movies.movieId.unique())

**Output:** menghasilkan jumlah data dari fix_movies adalah 59047.

Tahapan untuk melakukan pengecekan untuk kategori genres yang unik.

In [ ]:
# Mengecek kategori genres yang unik
fix_movies.genres.unique()

**Output:** menghasilkan berbagai macam genre dengan adanya kombinasi genre pada film.

Melakukan pengecekan kategori genres untuk data (no genres listed)

In [ ]:
# Mengecek kategori genres (no genres listed)
fix_movies[fix_movies['genres'] == '(no genres listed)']

**Output:** menunjukkan hasil data (no genres listed) pada kolom genres sebanyak 26627 baris dan 6 kolom.

Mengubah nama kategori genres  (no genres listed) menjadi ‘unknown’

In [ ]:
# Mengubah nama kategori genres  (no genres listed) menjadi ‘unknown’
fix_movies = fix_movies.replace('(no genres listed)', 'unknown')
fix_movies[fix_movies['genres'] == 'unknown']

**Output:** menunjukkan bahwa terjadi perubahan pada kolom genres di data (no genres listed) menjadi unknown.

Melakukan ganti karakter `|` menjadi koma dan menghapus karakter non-alphabet pada kolom genres

In [ ]:
fix_movies['genres'] = fix_movies['genres'].str.replace('|', ', ', regex=False).str.replace('[^a-zA-Z, ]', '', regex=True)

Melakukan pembuatan variabel preparation yang berisi dataframe fix_movies kemudian mengurutkan berdasarkan movieId.

In [ ]:
# Membuat variabel preparation yang berisi dataframe fix_movies kemudian mengurutkan berdasarkan movieId
preparation = fix_movies
preparation.sort_values('movieId')

**Output:** menunjukkan tabel data yang memiliki 6 kolom yaitu userId, movieId, rating, timestamp, dan title dengan jumlah baris sebanyak 25000095.

Menghapus data duplikat pada variabel preparation menggunakan fungsi `drop_duplicates` berdasarkan kolom movieId.

In [ ]:
# Membuang data duplikat pada variabel preparation
preparation = preparation.drop_duplicates('movieId')
preparation

**Output:** terjadinya perubahan jumlah baris, di mana sebelumnya berjumlah 25000095. Namun ketika menghapus duplikasi menjadi 59047 baris dan 6 kolom.

Melakukan proses konversi data series menjadi bentuk list menggunakan fungsi `tolist()`. Untuk data yang di konversi adalah data kolom movieId, title, dan genres.

In [ ]:
# Mengonversi data series ‘movieId’, ‘title’, dan ‘genres’ menjadi dalam bentuk list
movies_id = preparation['movieId'].tolist()
movies_title = preparation['title'].tolist()
movies_genres = preparation['genres'].tolist()

print(len(movies_id))
print(len(movies_title))
print(len(movies_genres))

**Output:** hasil menunjukkan bahwa untuk masing-masing data movies_id, movies_title, dan movies_genres memiliki jumlah yang sama yaitu 59047.

Melakukan pembuatan dictionary untuk menentukan pasangan key-value pada data movies_id, movies_title, dan movies_genres.

In [ ]:
# Membuat dictionary untuk data ‘movies_id’, ‘movies_title’, dan ‘movies_genres’
movies_new = pd.DataFrame({
    'id': movies_id,
    'movies_title': movies_title,
    'genres': movies_genres
})
movies_new

**Output:** menunjukkan tabel yang berisikan tiga kolom yaitu id yang berisikan movies_id, movies_title berisikan data movies_title, dan genres berisikan data movies_genres. Untuk jumlah baris datanya adalah 59047.

## **Model Development dengan Content Based Filtering**
Tahapan ini adalah melakukan pembangunan model sistem rekomendasi menggunakan pendekatan Content Based Filtering. Pendekatan ini dilakukan untuk mengetahui rekomendasi item yang mirip dengan item yang disukai pengguna.

Tahapan ini diawali dengan melakukan pengecekan data dan assign dataframe ke dalam variabel data.

In [ ]:
data = movies_new
data.sample(5)

**Output:** menunjukkan variabel data berdasarkan movies_new dengan menampilkan hanya 5 sampel data saja baik itu id, movies_title, dan genres.

### **TF-IDF Vectorizer**
Tahapan ini adalah untuk membangun sistem rekomendasi sederhana berdasarkan kategori genres pada film. Teknik ini untuk menemukan representasi fitur penting dari setiap kategori genres.

Tahapan ini diawali dengan proses inisialisasi TfidfVectorizer. Kemudian melakukan perhitungan idf pada data genres dan mapping array fitur index integer ke fitur nama. Proses ini menggunakan fungsi `tfidfvectorizer()` dari library sklearn.

In [ ]:
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()
tf.fit(data['genres'])

# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names_out()

**Output:** menunjukkan daftar dari genre film.

Tahapan untuk melakukan fit dan transformasi ke dalam bentuk matriks.

In [ ]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data['genres'])

# Melihat ukuran matrix tfidf
tfidf_matrix.shape

**Output:** menghasilkan matriks dengan ukuran (59047, 20), di mana 59047 adalah ukuran data dan 20 merupakan matriks kategori genre.

Mengubah vektor tf-idf dalam bentuk matriks dengan menggunakan fungsi `todense()`.

In [ ]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

**Output:** menghasilkan konversi tfidf matrix dengan menunjukkan bentuk bilangan representasi numerik.

Melakukan pembatasan ukuran data dengan hanya mengambil 1000 data film.

In [ ]:
tfidf_limit = tfidf_matrix[:1000]

Melakukan pembuatan dataframe agar dapat meninjau tfidf matrix dengan kolom berisikan genre, dan baris berisikan judul film atau movies_title.

In [33]:
pd.DataFrame(
    tfidf_limit.todense(),
    columns=tf.get_feature_names_out(),
    index=data.movies_title[:1000]
).sample(20, axis=1).sample(15, axis=0)

,romance,unknown,war,adventure,musical,drama,scifi,children,thriller,animation,action,documentary,mystery,western,filmnoir,fantasy,imax,crime,comedy,horror
movies_title,,,,,,,,,,,,,,,,,,,,
Billy's Holiday (1995),0.000000,0.0,0.0,0.000000,0.937338,0.348422,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
Once Upon a Time... When We Were Colored (1995),0.854149,0.0,0.0,0.000000,0.000000,0.520029,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
Beautiful Girls (1996),0.720554,0.0,0.0,0.000000,0.000000,0.438693,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.536982,0.000000
Yankee Zulu (1994),0.000000,0.0,0.0,0.000000,0.000000,0.632671,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.774421,0.000000
"Basketball Diaries, The (1995)",0.000000,0.0,0.0,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
Living in Oblivion (1995),0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,1.000000,0.000000
"Goofy Movie, A (1995)",0.450074,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.586340,0.000000,0.584071,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.335411,0.000000
Junior (1994),0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.856733,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.515760,0.000000
"Wes Craven's New Nightmare (Nightmare on Elm Street Part 7: Freddy's Finale, A) (1994)",0.000000,0.0,0.0,0.000000,0.000000,0.298099,0.000000,0.000000,0.468823,0.000000,0.000000,0.0,0.642582,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.527665


**Output:** menunjukkan film Maya Lin: A Strong Clear Vision (1994) memiliki kategori yaitu documentary. Hal ini terlihat dari nilai matriks 1.0 pada kategori comedy. Untuk film Basketball Diaries, The (1995) memiliki kategori drama.

### **Cosine Similarity**
Tahapan ini dilakukan untuk menghitung derajat kesamaan (similarity degree) dengan teknik cosine similarity.

Melakukan perhitungan cosine similarity pada matrix tf-idf menggunakan fungsi `cosine_similarity` pada data `tfidf_limit`.

In [ ]:
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_limit)
cosine_sim

**Output:** menghasilkan keluaran berupa matriks kesamaan dalam bentuk array.

Melakukan peninjauan matriks kesamaan setiap movie dengan menampilkan judul movie dalam 5 sampel kolom (axis = 1) dan 9 sampel baris (axis=0)

In [35]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa nama movie
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['movies_title'][:1000], columns=data['movies_title'][:1000])
print('Shape:', cosine_sim_df.shape)

# Melihat similarity matrix pada setiap movie
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (1000, 1000)


movies_title,To Cross the Rubicon (1991),Anne Frank Remembered (1995),Mr. Jones (1993),Harriet the Spy (1996),Maya Lin: A Strong Clear Vision (1994)
movies_title,,,,,
"Cowboy Way, The (1994)",0.434648,0.0,0.226030,0.264175,0.0
"Ghost and Mrs. Muir, The (1947)",0.344181,0.0,0.661849,0.000000,0.0
Mr. Wonderful (1993),0.000000,0.0,0.684882,0.296708,0.0
Shallow Grave (1994),0.448484,0.0,0.233225,0.272584,0.0
D3: The Mighty Ducks (1996),0.000000,0.0,0.000000,1.000000,0.0
Barb Wire (1996),0.000000,0.0,0.000000,0.000000,0.0
Beyond Bedlam (1993),0.491874,0.0,0.255789,0.000000,0.0
Death in Brunswick (1991),0.000000,0.0,0.000000,0.496540,0.0
Surviving the Game (1994),0.000000,0.0,0.000000,0.000000,0.0


**Output:** berhasil mengidentifikasi kesamaan antara satu film dengan film lainnya. Shape yang digunakan yaitu (1000, 1000) merupakan ukuran matriks similarity dari data yang dimiliki. Berdasarkan data yang ada, matriks di atas sebenarnya berukuran 1000 judul film x 1000 judul film (masing-masing dalam sumbu X dan Y). Artinya, mengidentifikasi tingkat kesamaan pada 1000 judul film. Hanya memilih 10 judul film pada baris vertikal dan 5 judul film pada sumbu horizontal. Diketahui bahwa angka 1.0 mengindikasikan bahwa film pada kolom X (horizontal) memiliki kesamaan dengan film pada baris Y (vertikal). Hal tersebut dapat dilihat pada film D3: The Mighty Ducks (1996) yang teridentifikasi sama (similar) dengan film Harriet the Spy (1996).

### **Mendapatkan Rekomendasi**
Tahapan untuk melakukan proses rekomendasi film.

Pada tahapan ini membuat fungsi `def movies_recommendations` yang terdiri dari beberapa parameter sebagai berikut:
- Judul_film: judul_film (index kemiripan dataframe).
- Similarity_data : Dataframe mengenai similarity yang telah kita definisikan sebelumnya yaitu `cosine_sim_df`.
- Items : judul dan genre yang digunakan untuk mendefinisikan kemiripan, dalam hal ini adalah `movies_title` dan `genres`.
- k : Banyak rekomendasi yang ingin diberikan yaitu berjumlah 5.

In [ ]:
def movies_recommendations(judul_film, similarity_data=cosine_sim_df, items=data[['movies_title', 'genres']], k=5):

    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,judul_film].to_numpy().argpartition(
        range(-1, -k, -1))

    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]

    # Drop judul_film agar nama movie yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(judul_film, errors='ignore')

    return pd.DataFrame(closest).merge(items).head(k)

Implementasi kode `def movies_recommendation` untuk menemukan rekomendasi film yang mirip dengan `Toy Story (1995)`.

In [37]:
data[data.movies_title.eq('Toy Story (1995)')]

,id,movies_title,genres
0,1,Toy Story (1995),"Adventure, Animation, Children, Comedy, Fantasy"


**Output:** menunjukkan tabel dengan judul film `Toy Story (1995)` dengan genre `Adventure, Animation, Children, Comedy, Fantasy`.

Mendapatkan movie recommendation dengan memanggil fungsi `movies_recommendation`.

In [38]:
movies_recommendations('Toy Story (1995)')

,movies_title,genres
0,Space Jam (1996),"Adventure, Animation, Children, Comedy, Fantas..."
1,"Pagemaster, The (1994)","Action, Adventure, Animation, Children, Fantasy"
2,Kids of the Round Table (1995),"Adventure, Children, Comedy, Fantasy"
3,"NeverEnding Story III, The (1994)","Adventure, Children, Fantasy"
4,Jumanji (1995),"Adventure, Children, Fantasy"


**Output:** memberikan rekomendasi 5 nama film dengan kategori `genres` yaitu Action, Adventure, Animation, Children, Comedy, dan Fantasy.

## Evaluation
Tahap ini untuk melakukan proses evaluasi denggan menggunakan perhitungan akurasi.

Proses evaluasi ini menggunakan precision@k untuk mengukur kesamaan item pada hasil rekomendasi teratas atau k rekomendasi menggunakan fungsi `def precision_at_k_movies`.

In [ ]:
def precision_at_k_movies(movie_title, k=5):
    recommendation = movies_recommendations(movie_title, k=k)
    k_recommendation = recommendation['movies_title'].tolist()
    relevant_count = len([title for title in k_recommendation if title in data['movies_title'].values])
    precision = relevant_count / k
    return precision

Melakukan print hasil akurasi dari precision@k dari judul film Toy Story (1995) berdasarkan k=5.

In [ ]:
print(precision_at_k_movies('Toy Story (1995)', k=5))

**Output:** sistem berhasil enunjukkan bahwa 5 film hasil rekomendasi teratas memiliki genre yang mirip dengan film Toy Story (1995) di mana dapat dilihat pada hasil akurasi precision@k yang memiliki nilai 1.0.